In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Epilepsy/GSE205661'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrated analysis of expression profile and potential pathogenic mechanism of temporal lobe epilepsy with hippocampal sclerosis"
!Series_summary	"To investigate the potential pathogenic mechanism of temporal lobe epilepsy with hippocampal sclerosis (TLE+HS), we have employed analyzing of the expression profiles of microRNA/ mRNA/ lncRNA/ DNA methylation in brain tissues of hippocampal sclerosis (TLE+HS) patients. Brain tissues of six patients with TLE+HS and nine of normal temporal or parietal cortices (NTP) of patients undergoing internal decompression for traumatic brain injury (TBI) were collected. The total RNA was dephosphorylated, labeled, and hybridized to the Agilent Human miRNA Microarray, Release 19.0, 8x60K. The cDNA was labeled and hybridized to the Agilent LncRNA+mRNA Human Gene Expression Microarray V3.0，4x180K. For methylation detection, the DNA was labeled and hybridized to the Illumina 450K Infinium Methylation BeadChip. The raw

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True

# Variable Availability and Data Type Conversion
# Step 2.1: Data Availability
trait_row = 0 if len({x.split(': ')[1] for x in {0: ['tissue: Hippocampus', 'tissue: Temporal lobe', 'tissue: Parietal lobe']}[0]}) > 1 else None
age_row = 2 if len({x.split(': ')[1] for x in {2: ['age: 23y', 'age: 29y', 'age: 37y', 'age: 26y', 'age: 16y', 'age: 13y', 'age: 62y', 'age: 58y', 'age: 63y', 'age: 68y', 'age: 77y', 'age: 59y', 'age: 50y', 'age: 39y']}[2]}) > 1 else None
gender_row = 1 if len({x.split(': ')[1] for x in {1: ['gender: Female', 'gender: Male']}[1]}) > 1 else None

# Step 2.3: Data Type Conversion
def convert_trait(value):
    if trait_row is None:
        return None
    val = value.split(': ')[1]
    return 1 if val == 'Hippocampus' else 0 if val in ['Temporal lobe', 'Parietal lobe'] else None

def convert_age(value):
    if age_row is None:
        return None
    val = value.split(': ')[1]
    try:
        return int(val[:-1])  # remove 'y' from age and convert to integer
    except ValueError:
        return None

def convert_gender(value):
    if gender_row is None:
        return None
    val = value.split(': ')[1]
    return 1 if val == 'Male' else 0 if val == 'Female' else None

# Save Metadata
save_cohort_info('GSE205661', './preprocessed/Epilepsy/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Epilepsy', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Epilepsy/trait_data/GSE205661.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM6216198': [1, 23, 0], 'GSM6216199': [1, 29, 1], 'GSM6216200': [1, 37, 1], 'GSM6216201': [1, 26, 0], 'GSM6216202': [1, 16, 0], 'GSM6216203': [1, 13, 1], 'GSM6216204': [0, 62, 0], 'GSM6216205': [0, 58, 1], 'GSM6216206': [0, 63, 0], 'GSM6216207': [0, 68, 1], 'GSM6216208': [0, 77, 1], 'GSM6216209': [0, 59, 1], 'GSM6216210': [0, 50, 1], 'GSM6216211': [0, 39, 1], 'GSM6216212': [0, 23, 1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236',
       'cg00000289', 'cg00000292', 'cg00000321', 'cg00000363', 'cg00000622',
       'cg00000658', 'cg00000714', 'cg00000721', 'cg00000734', 'cg00000769',
       'cg00000807', 'cg00000884', 'cg00000905', 'cg00000924', 'cg00000948'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Name': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'AddressA_ID': ['31729416', '32735311', '28780415', '16712347', '19779393'], 'AlleleA_ProbeSeq': ['AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAAATTC', 'ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAACCCCA', 'AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTCCATC', 'TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTCCATC', 'AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAACTCTC'], 'AddressB_ID': [nan, '31717405', nan, nan, nan], 'AlleleB_ProbeSeq': [nan, 'ACGAAAAAACAACGCACAACTATAATAATTTTTAAAATAAATAAACCCCG', nan, nan, nan], 'Infinium_Design_Type': ['II', 'I', 'II', 'II', 'II'], 'Next_Base': [nan, 'A', nan, nan, nan], 'Color_Channel': [nan, 'Red', nan, nan, nan], 'Forward_Sequence': ['AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCTGCTCAGCTGGAAAC[CG]AATTTGAGATAAATTCAAGGGTCTATGTGGACAAGACTGCTAGTGTCTCTCTCTGGATTG', 'TATCTCTGTCT

### Step 6: Gene Identifier Mapping

In [7]:
# Let's try using additional columns or another source as identifier_key

# Possible keys to check: 'ID', 'Name', or others seen in preview
identifier_key = 'ID'
gene_symbol_key = 'UCSC_RefGene_Name'

print("Available Columns in Gene Annotation:")
print(gene_annotation.columns)

# Try alternative identifiers
possible_identifier_keys = ['ID', 'Name']
for identifier_key in possible_identifier_keys:
    print(f"Trying Identifier Key: {identifier_key}")

    # 2. Get the dataframe storing the mapping between probe IDs and genes
    gene_mapping_df = get_gene_mapping(gene_annotation, prob_col=identifier_key, gene_col=gene_symbol_key)

    # Validate the mapping dataframe content
    print("Gene Mapping DataFrame Preview:")
    print(preview_df(gene_mapping_df))

    # 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
    gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping_df)

    # Verify the resulting gene_data preview for any issues
    print("Mapped Gene Data Preview:")
    print(preview_df(gene_data_mapped))

    # If the mapping is successful, reassign gene_data
    if not gene_data_mapped.empty:
        gene_data = gene_data_mapped
        break
    else:
        print(f"Mapping with {identifier_key} resulted in an empty dataframe, trying next key...")

# Final validation
print("Final Mapped Gene Data Preview:")
print(preview_df(gene_data))


Available Columns in Gene Annotation:
Index(['ID', 'Name', 'AddressA_ID', 'AlleleA_ProbeSeq', 'AddressB_ID',
       'AlleleB_ProbeSeq', 'Infinium_Design_Type', 'Next_Base',
       'Color_Channel', 'Forward_Sequence', 'Genome_Build', 'CHR', 'MAPINFO',
       'SourceSeq', 'Chromosome_36', 'Coordinate_36', 'Strand', 'Probe_SNPs',
       'Probe_SNPs_10', 'Random_Loci', 'Methyl27_Loci', 'UCSC_RefGene_Name',
       'UCSC_RefGene_Accession', 'UCSC_RefGene_Group', 'UCSC_CpG_Islands_Name',
       'Relation_to_UCSC_CpG_Island', 'Phantom', 'DMR', 'Enhancer',
       'HMM_Island', 'Regulatory_Feature_Name', 'Regulatory_Feature_Group',
       'DHS', 'RANGE_START', 'RANGE_END', 'RANGE_GB', 'SPOT_ID'],
      dtype='object')
Trying Identifier Key: ID


Gene Mapping DataFrame Preview:
{'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Gene': ['TTTY18', 'TSPY4;FAM197Y2', 'DAZ1;DAZ4;DAZ4', 'EIF1AY', 'BCORL2']}


Mapped Gene Data Preview:
{'GSM6216198': [0.498235294117647, 0.9092857142857143, 0.6978517520215634, 0.8495714285714285, 0.8033571428571429], 'GSM6216199': [0.479764705882353, 0.9005714285714286, 0.7023692722371968, 0.8647857142857143, 0.8315714285714285], 'GSM6216200': [0.4908823529411765, 0.8999999999999999, 0.6921832884097036, 0.8464285714285714, 0.8146428571428571], 'GSM6216201': [0.5277647058823529, 0.9008571428571428, 0.6847358490566038, 0.8594285714285714, 0.7719285714285714], 'GSM6216202': [0.48570588235294115, 0.8958571428571428, 0.6805498652291105, 0.8203571428571428, 0.7764285714285714], 'GSM6216203': [0.4645294117647059, 0.8968571428571428, 0.67955525606469, 0.8494285714285714, 0.7889285714285714], 'GSM6216204': [0.5437647058823529, 0.9052857142857142, 0.6680485175202157, 0.8423571428571428, 0.7965], 'GSM6216205': [0.5305294117647059, 0.8950000000000001, 0.6788436657681941, 0.8602142857142857, 0.8215714285714286], 'GSM6216206': [0.5418823529411765, 0.9201428571428572, 0.676

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Epilepsy/gene_data/GSE205661.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Epilepsy')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE205661', './preprocessed/Epilepsy/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Epilepsy/GSE205661.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Epilepsy', the least common label is '1.0' with 6 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.

Quartiles for 'Age':
  25%: 24.5
  50% (Median): 39.0
  75%: 60.5
Min: 13.0
Max: 77.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 5 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

